<a href="https://colab.research.google.com/github/IsabellaAGomes/redundancy_identification/blob/main/step_3_separacao_de_conjuntos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input_file_name = "assin2-pos.xml"
output_training_file_name = "assin2-trv1-pos.xml"
output_testing_file_name = "assin2-t1-pos.xml"
output_training_undersamplingA_file_name = "assin2-trv1-pos_undersampling_A.xml"
output_training_undersamplingB_file_name = "assin2-trv1-pos_undersampling_B.xml"
output_training_oversamplingA_file_name = "assin2-trv1-pos_oversampling_A.xml"
output_training_oversamplingB_file_name = "assin2-trv1-pos_oversampling_B.xml"


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!python --version ou !python -V

Python 3.10.12


In [ ]:
#Importando bibliotecas padrões + módulos de classificação da bibloteca do Bert, módulo de tokenização e módulo de pipeline
import pandas as pd
import numpy as np
print(np.__version__)
print(pd.__version__)

1.25.2
1.5.3


In [ ]:
df = pd.read_xml(input_file_name)

df['redundancy'] = np.where(df['similarity'] < 4, 0, 1)

In [ ]:
df_classe_0 = df.loc[df['redundancy'] == 0]
df_classe_1 = df.loc[df['redundancy'] == 1]

#Divisão do conjunto de forma aleatória (random_state = 25) separando 80% do modelo para treino e o restante para teste
training_classe_0_df = df_classe_0.sample(frac=0.8, random_state=25)
testing_classe_0_df = df_classe_0.drop(training_classe_0_df.index)
training_classe_1_df = df_classe_1.sample(frac=0.8, random_state=25)
testing_classe_1_df = df_classe_1.drop(training_classe_1_df.index)

#Juntando os conjuntos de classe 0 e 1
frames_training = [training_classe_0_df, training_classe_1_df]
training_df = pd.concat(frames_training)
training_df = training_df.sample(frac = 1)
#n = np.sum(len(df.columns) for df in frames_training)
#training_df = pd.concat(frames_training, axis=1).iloc[:, random.sample(range(n), n)]

frames_testing = [testing_classe_0_df, testing_classe_1_df]
testing_df = pd.concat(frames_testing)
testing_df = testing_df.sample(frac = 1)
#n = np.sum(len(df.columns) for df in frames_testing)
#esting_df = pd.concat(frames_testing, axis=1).iloc[:, random.sample(range(n), n)]


In [ ]:
#A - Undersampling para o ASSIN (3505)
# Classe 0 = 1577 (45%)
# Classe 1 = 1928 (55%)
training_classe_0_df_undersampling_A = training_classe_0_df.sample(frac=0.26, random_state=25)
frames_training = [training_classe_0_df_undersampling_A, training_classe_1_df]
training_df_undersampling_A = pd.concat(frames_training)
training_df_undersampling_A = training_df_undersampling_A.sample(frac = 1)

#B - Undersampling para o ASSIN (5508)
# Classe 0 = 3580 (65%)
# Classe 1 = 1928 (35%)
training_classe_0_df_undersampling_B = training_classe_0_df.sample(frac=0.59, random_state=25)
frames_training = [training_classe_0_df_undersampling_B, training_classe_1_df]
training_df_undersampling_B = pd.concat(frames_training)
training_df_undersampling_B = training_df_undersampling_B.sample(frac = 1)


#A - Oversampling para o ASSIN (13493)
# Classe 0 = 6072 (45%)
# Classe 1 = 7421 (55%)
training_classe_1_df_copy_A = training_classe_1_df
training_classe_1_df_copy_B = training_classe_1_df
training_classe_1_df_oversampling_A = training_classe_1_df.sample(frac=0.85, random_state=25)
frames_training = [training_classe_0_df, training_classe_1_df, training_classe_1_df_copy_A, training_classe_1_df_copy_B, training_classe_1_df_oversampling_A]
training_df_oversampling_A = pd.concat(frames_training)
training_df_oversampling_A = training_df_oversampling_A.sample(frac = 1)


#B - Oversampling para o ASSIN (9341)
# Classe 0 = 6072 (65%)
# Classe 1 = 3269 (35%)
training_classe_1_df_oversampling_B = training_classe_1_df.sample(frac=0.7, random_state=25)
frames_training = [training_classe_0_df, training_classe_1_df, training_classe_1_df_oversampling_B]
training_df_oversampling_B = pd.concat(frames_training)
training_df_oversampling_B = training_df_oversampling_B.sample(frac = 1)



In [ ]:
#### UNDERSAMPLING E OVERSAMPLING C FORAM DESCARTADOS PORQUE COM O B JÁ É POSSÍVEL OBTER BONS RESULTADOS.

#C - Undersampling para o ASSIN (4284)
# Classe 0 = 2356 (55%)
# Classe 1 = 1928 (45%)
#training_classe_0_df_undersampling_C = training_classe_0_df.sample(frac=0.39, random_state=25)
#frames_training = [training_classe_0_df_undersampling_C, training_classe_1_df]
#training_df_undersampling_C = pd.concat(frames_training)
#training_df_undersampling_C = training_df_undersampling_C.sample(frac = 1)

#C - Oversampling para o ASSIN (11040)
# Classe 0 = 6072 (55%)
# Classe 1 = 4968 (45%)
#training_classe_1_df_copy_A = training_classe_1_df
#training_classe_1_df_oversampling_C = training_classe_1_df.sample(frac=0.58, random_state=25)
#frames_training = [training_classe_0_df, training_classe_1_df, training_classe_1_df_copy_A, training_classe_1_df_oversampling_C]
#training_df_oversampling_C = pd.concat(frames_training)
#training_df_oversampling_C = training_df_oversampling_C.sample(frac = 1)


In [ ]:
#Divisão do conjunto de forma aleatória (random_state = 25) separando 80% do modelo para treino e o restante para teste
#training_df = df.sample(frac=0.8, random_state=25).reset_index(drop=True)
#testing_df = df.drop(training_df.index).reset_index(drop=True)

In [ ]:
training_df_undersampling_A.to_xml(output_training_undersamplingA_file_name)
training_df_undersampling_B.to_xml(output_training_undersamplingB_file_name)
#training_df_undersampling_C.to_xml('assin-trv1-pos_undersampling_C.xml')
training_df_oversampling_A.to_xml(output_training_oversamplingA_file_name)
training_df_oversampling_B.to_xml(output_training_oversamplingB_file_name)
#training_df_oversampling_C.to_xml('assin-trv1-pos_oversampling_C.xml')

In [ ]:
training_df.to_xml(output_training_file_name)
testing_df.to_xml(output_testing_file_name )
